In [1]:
import os
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tabulate import tabulate
import pandas as pd
# np.set_printoptions(threshold=np.inf)


C:\Users\ariap\AppData\Local\Temp\ipykernel_17028\1468199686.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Import module yang digunkaan
from preprocessing.scarpping_component import extract_component_by_images
from helper.helper import format_number_and_round
from feature_extraction.poc import POC as POC
from feature_extraction.get_vektor import getVektor as Vect
from feature_extraction.get_quadran import getQuadran as Qd

# load model dan shape predictor untuk deteksi wajah
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

components_setup = {
    'mulut': {
        'object_name': 'mouth',
        'object_rectangle': {"x_right": 54, "x_left": 48, "y_highest": 52, "y_lowest": 57},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 5}
    },
    'mata_kiri': {
        'object_name': 'eye_left',
        'object_rectangle': {"x_right": 39, "x_left": 36, "y_highest": 38, "y_lowest": 41},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 25}
    },
    'mata_kanan': {
        'object_name': 'eye_right',
        'object_rectangle': {"x_right": 45, "x_left": 42, "y_highest": 43, "y_lowest": 47},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 25}
    },
    'alis_kiri': {
        'object_name': 'eyebrow_left',
        'object_rectangle': {"x_right": 21, "x_left": 17, "y_highest": 18, "y_lowest": 21},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15}
    },
    'alis_kanan': {
        'object_name': 'eyebrow_right',
        'object_rectangle': {"x_right": 26, "x_left": 22, "y_highest": 25, "y_lowest": 22},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15}
    }
}

# Inisialisasi variabel untuk path url dan blok size
pathDatasetImage = "dataset/sub01/EP02_01f"
blockSize = 7

# Inisialisasi variabel untuk menyimpan data dari masing-masing komponen
data_blocks_first_image = {component_name: None for component_name in components_setup}
frames_data = {component_name: [] for component_name in components_setup}
index = {component_name: 0 for component_name in components_setup}

# looping semua file yang ada didalam
for filename in os.listdir(pathDatasetImage):
    if filename.endswith(".jpg") or filename.endswith(".png"): 
        # Create the directory if it doesn't exist
        image = cv2.imread(os.path.join(pathDatasetImage, filename))
        image = cv2.resize(image, (600, 500))
        # image = cv2.resize(image, (98, 56))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # detect the faces
        rects = detector(gray)

        # Inisialisasi data per frame
        frame_data = {'Frame': f"{index + 1}({filename.split('.')[0]})"}

        # Inisialisasi data untuk setiap blok
        for i in range(120):  # Jumlah blok maksimum 120
            frame_data[f'X{i+1}'] = 0
            frame_data[f'Y{i+1}'] = 0
            frame_data[f'Tetha{i+1}'] = 0
            frame_data[f'Magnitude{i+1}'] = 0

        # Memproses rects untuk setiap bentuk wajah yang terdeteksi
        for rect in rects:
            # Ambil bentuk wajah dalam bentuk shape sesuai dengan model predictor
            shape = predictor(gray, rect)
            # Memproses setiap komponen wajah
            for component_name, component_info in components_setup.items():
                # Ambil data blok image dari return fungsi extract_component_by_images
                data_blocks_image_current = np.array(extract_component_by_images(
                    image=image,
                    shape=shape,
                    frameName=filename.split(".")[0],
                    objectName=component_info['object_name'],
                    objectRectangle=component_info['object_rectangle'],
                    pixelShifting=component_info['pixel_shifting'],
                ))
                
                # Ambil frame pertama dari perulangan lalu simpan di variabel dan skip (lanjutkan ke frame berikut)
                if data_blocks_first_image[component_name] is None:
                    data_blocks_first_image[component_name] = data_blocks_image_current
                    continue

                # Tampilkan 
                plt.imshow(np.uint8(data_blocks_image_current), cmap="gray")

                # Inisiasi class POC
                initPOC = POC(data_blocks_first_image[component_name], data_blocks_image_current, blockSize) 
                # Pemanggilan fungsi pocCalc() untuk menghitung nilai POC disetiap gambar
                valPOC = initPOC.getPOC() 

                # Pemanggilan class dan method untuk menampilkan quiver / gambar panah
                initQuiv = Vect(valPOC, blockSize)
                quivData = initQuiv.createVektor() 

                plt.quiver(quivData[:, 0], quivData[:, 1], quivData[:, 2], quivData[:, 3], scale=1, scale_units='xy', angles='xy', color="r")    

                # num = 0
                for rect_def in valPOC[2]:
                    x, y, width, height = rect_def
                    
                    rects = patches.Rectangle((x,y), width,height, edgecolor='r', facecolor='none') 
                    plt.gca().add_patch(rects)
                    
                    # plt.text(x,y,f'({num})', color="blue") 
                    # num += 1

                # Pemanggilan class untuk mengeluarkan nilai karakteristik vektor
                # blok ke, x,y,tetha, magnitude, dan quadran ke
                initQuadran = Qd(quivData)
                quadran = initQuadran.setQuadran()

                print(tabulate(quadran, headers=['Blok Ke', 'X', 'Y', 'Tetha', 'Magnitude', 'Quadran Ke']))
                plt.axis('on')  # Matikan sumbu
                plt.show()     
            
                # Update data frame dengan data quadran
                for i, quad in enumerate(quadran):
                    frame_data[f'X{i+1}'] = str(format_number_and_round(quad[1]))
                    frame_data[f'Y{i+1}'] = str(format_number_and_round(quad[2]))
                    frame_data[f'Tetha{i+1}'] = str(format_number_and_round(quad[3]))
                    frame_data[f'Magnitude{i+1}'] = str(format_number_and_round(quad[4]))

        component_frames_data = []
        for component_name, component_data in data_blocks.items():
            component_frame_data = {'Component': component_name}
            for i, component_block_data in enumerate(component_data):
                component_frame_data[f'Block {i+1}'] = component_block_data
            component_frames_data.append(component_frame_data)
            
        # Append data frame ke list
        frames_data.extend(frame_data)
        # tambah index disetiap looping frame
        index += 1
            
# Buat DataFrame dari list data frame
df = pd.DataFrame(frames_data)



# Remove file output.csv
if os.path.exists('output.csv') and os.path.exists('output.xlsx'):
    os.remove('output.csv')
    os.remove('output.xlsx')
if os.path.exists('output.csv'):
    os.remove('output.csv')
if os.path.exists('output.xlsx'):
    os.remove('output.xlsx')
# print(df.to_string())

# Export dataframe ke dalam file excel xlsx
df.to_excel('output.xlsx', index=False, float_format=None)

# Export DataFrame ke dalam file CSV
df.to_csv('output.csv', index=False, float_format=None)
# df.to_csv('output.csv', sep=',', float_format=None, encoding='utf-8')

ModuleNotFoundError: No module named 'feature_extraction.get_vektor'